In [44]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()

In [166]:
# download google coordinate file
import requests

from shapely.geometry import mapping, shape
from shapely.prepared import prep

data = requests.get("https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson").json()

# WORLD dataframe uses ISO_A3, so we extract that one from the data
iso_a3_lookup = {}
for feature in data['features']:
    iso_a3_lookup[feature['properties']['ISO_A3']] = prep(shape(feature['geometry']))
    
def get_iso(point):
    for country, geom in iso_a3_lookup.items():
        if geom.contains(point):
            return country
    return 'unknown'

In [176]:
df = pd.read_csv('covid_19_data.csv')
df.columns = [i.replace('/', '').replace(' ', '') for i in df.columns]
df.ObservationDate = pd.to_datetime(df.ObservationDate, format='%m/%d/%Y')
df.ProvinceState.replace('None', np.nan, inplace=True)
df.ProvinceState.fillna(df.CountryRegion, inplace=True)
df.ProvinceState = df.ProvinceState.apply(lambda x: x.strip())
df.CountryRegion = df.CountryRegion.apply(lambda x: x.strip())
if len(df['ProvinceState'].drop_duplicates()) == len(df[['ProvinceState', 'CountryRegion']].drop_duplicates()):
    print('Provinces are unique and can be used as identifier')
else:
    print('Unfortunately, Provinces are not unique.')
    
# unique land identifier
df['ULID'] = df.ProvinceState + ' - ' + df.CountryRegion
df.sample(10)

Unfortunately, Provinces are not unique.


,SNo,ObservationDate,ProvinceState,CountryRegion,LastUpdate,Confirmed,Deaths,Recovered,ULID
4695,4696,2020-03-10,Panama,Panama,2020-03-10T06:13:43,1.0,0.0,0.0,Panama - Panama
1172,1173,2020-02-10,Italy,Italy,2020-02-07T17:53:02,3.0,0.0,0.0,Italy - Italy
1300,1301,2020-02-12,Thailand,Thailand,2020-02-11T18:43:04,33.0,0.0,10.0,Thailand - Thailand
4697,4698,2020-03-10,Saint Barthelemy,Saint Barthelemy,2020-03-09T10:43:06,1.0,0.0,0.0,Saint Barthelemy - Saint Barthelemy
2396,2397,2020-02-26,Hebei,Mainland China,2020-02-26T10:33:02,312.0,6.0,261.0,Hebei - Mainland China
141,142,2020-01-25,Liaoning,Mainland China,1/25/20 17:00,17.0,0.0,0.0,Liaoning - Mainland China
3289,3290,2020-03-04,United Arab Emirates,United Arab Emirates,2020-03-03T23:43:02,27.0,0.0,5.0,United Arab Emirates - United Arab Emirates
3262,3263,2020-03-04,Hainan,Mainland China,2020-03-04T10:03:19,168.0,5.0,158.0,Hainan - Mainland China
1056,1057,2020-02-09,Anhui,Mainland China,2020-02-09T09:33:02,779.0,1.0,72.0,Anhui - Mainland China
2149,2150,2020-02-23,Qinghai,Mainland China,2020-02-21T04:43:02,18.0,0.0,18.0,Qinghai - Mainland China


In [194]:
# this DF rewrites the one above as a time series, but it also contains lat/long data
# use it to create a lat/long data lookup
df_conf = pd.read_csv('time_series_covid_19_confirmed.csv')
df_conf.columns = [i.replace('/', '').replace(' ', '') for i in df_conf.columns]
df_conf.ProvinceState.replace('None', np.nan, inplace=True)
df_conf.ProvinceState.fillna(df_conf.CountryRegion, inplace=True)
df_conf.ProvinceState = df_conf.ProvinceState.apply(lambda x: x.strip())
df_conf.CountryRegion = df_conf.CountryRegion.apply(lambda x: x.strip())
df_conf['ULID'] = df_conf.ProvinceState + ' - ' + df_conf.CountryRegion
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(df_conf['Long'], df_conf['Lat'])]
coord_lookup = dict(zip(df_conf.ULID, geometry))

### Use the coordinates provided in the dataset to replace place names with ISO names from Google API

In [203]:
iso_lookup = {}
for name, point in coord_lookup.items():
    iso_lookup[name] = get_iso(point)
for j in [i for i in df.ULID.unique() if i not in df_conf.ULID.values]:
    if j.endswith('US'):
        iso_lookup[j] = 'USA'
    elif j.endswith('Canada'):
        iso_lookup[j] = 'CAN'
iso_lookup['Australia - Australia'] = 'AUS'
iso_lookup['Ivory Coast - Ivory Coast'] = 'CIV'
iso_lookup['Bavaria - Germany'] = 'DEU'
iso_lookup['Cruise Ship - Others'] = 'unknown'
iso_lookup['From Diamond Princess - Israel'] = 'ISR'
iso_lookup['North Ireland - North Ireland'] = 'GBR'
iso_lookup['Republic of Ireland - Republic of Ireland'] = 'IRL'
iso_lookup['Hong Kong - Hong Kong'] = 'HKG'
iso_lookup['Philippines - Philippines'] = 'PHL'
iso_lookup['Cruise Ship - Others'] = 'unknown'
iso_lookup['New Zealand - New Zealand'] = 'NZL'
iso_lookup['Northern Territory - Australia'] = 'AUS'
iso_lookup['Faroe Islands - Faroe Islands'] = 'FRO'
iso_lookup['Grand Princess Cruise Ship - US'] = 'USA'
iso_lookup['Maldives - Maldives'] = 'MDV'
iso_lookup['Cyprus - Cyprus'] = 'CYP'
iso_lookup['Hong Kong - Hong Kong SAR'] = 'HKG'
iso_lookup['Grand Princess - US'] = 'USA'
iso_lookup['Hawaii - US'] = 'USA'
iso_lookup['Channel Islands - Channel Islands'] = 'GB'

In [204]:
df['iso_a3'] = df.ULID.apply(lambda x: iso_lookup[x])
df.sample(10)

,SNo,ObservationDate,ProvinceState,CountryRegion,LastUpdate,Confirmed,Deaths,Recovered,ULID,iso_a3
3553,3554,2020-03-05,"Maricopa County, AZ",US,2020-03-03T17:33:02,1.0,0.0,0.0,"Maricopa County, AZ - US",USA
1353,1354,2020-02-13,Beijing,Mainland China,2020-02-13T14:13:06,366.0,3.0,69.0,Beijing - Mainland China,CHN
2154,2155,2020-02-23,"Omaha, NE (From Diamond Princess)",US,2020-02-22T23:03:09,11.0,0.0,0.0,"Omaha, NE (From Diamond Princess) - US",USA
3925,3926,2020-03-07,Cameroon,Cameroon,2020-03-06T15:43:02,1.0,0.0,0.0,Cameroon - Cameroon,CMR
1667,1668,2020-02-17,Singapore,Singapore,2020-02-17T14:33:04,77.0,0.0,24.0,Singapore - Singapore,SGP
1217,1218,2020-02-11,Shanxi,Mainland China,2020-02-11T23:33:02,124.0,0.0,30.0,Shanxi - Mainland China,CHN
1316,1317,2020-02-12,India,India,2020-02-03T21:43:02,3.0,0.0,0.0,India - India,IND
1570,1571,2020-02-16,Anhui,Mainland China,2020-02-16T08:33:02,962.0,6.0,255.0,Anhui - Mainland China,CHN
260,261,2020-01-27,Malaysia,Malaysia,1/27/20 23:59,4.0,0.0,0.0,Malaysia - Malaysia,MYS
158,159,2020-01-25,Washington,US,1/25/20 17:00,1.0,0.0,0.0,Washington - US,USA
